In [1]:
import pandas as pd
import numpy as np


In [93]:
import mysql.connector

def load_stockdata(symbol):
    serverName = "localhost"
    cnx = mysql.connector.connect(user='michael', password='Password2017',
                                  host=serverName, database='stock_market')
    cursor = cnx.cursor()
    
    query = \
    "SELECT stock.*, bb.`sma20` AS bb_middle, bb.`band_upper` AS bb_upper, bb.`band_lower` AS bb_lower, \
    kc.`ema20` AS kc_middle, kc.`atr2_upper` AS kc_2_upper, kc.`atr2_lower` AS kc_2_lower, \
    kc.`atr3_upper` AS kc_3_upper, kc.`atr3_lower` AS kc_3_lower \
    FROM \
    ( SELECT * FROM (SELECT symbol, effective_date, `adj_open`, `adj_high`, `adj_low`, `adj_close`, `adj_volume` FROM `market.stock_price` \
      WHERE symbol = '{0}' ORDER BY effective_date DESC LIMIT 300) sub ORDER BY `effective_date` ASC ) stock \
    LEFT JOIN \
    `idc.bollinger_bands` AS bb ON bb.symbol = stock.symbol AND bb.effective_date = stock.effective_date \
    LEFT JOIN \
    `idc.keltnerchannels` AS kc ON kc.symbol = stock.symbol AND kc.effective_date = stock.effective_date".format(symbol)

    cursor.execute(query)
    dfsql = pd.DataFrame(cursor.fetchall(), dtype=np.float)
    dfsql.columns = cursor.column_names
    dfsql.set_index('effective_date', inplace=True)
    return dfsql


In [4]:
def drawChart(df):
    ax = df[["close"]].plot.line(grid='true', figsize=(18,10), title='Close chart', 
                                          marker=".", markersize=10, fontsize=18)
    ax.set_xlabel('tenor', fontsize=18)

In [31]:
import plotly.plotly as py
import plotly.graph_objs as go
from datetime import datetime

import plotly
plotly.offline.init_notebook_mode(connected=True) 

In [32]:
def drawInterChart(df):
    trace = go.Ohlc(x=df.date, open=df.open, high=df.high, low=df.low, close=df.close)
    data = [trace]
    layout = {
        'title': 'Stock Chart',
        'yaxis': {'title': 'Price'},
        'shapes': [{
            'x0': '2016-12-09', 'x1': '2016-12-09',
            'y0': 0, 'y1': 1, 'xref': 'x', 'yref': 'paper',
            'line': {'color': 'rgb(30,30,30)', 'width': 1}
        }],
        'annotations': [{
            'x': '2016-12-09', 'y': 0.05, 'xref': 'x', 'yref': 'paper',
            'showarrow': False, 'xanchor': 'left',
            'text': 'Increase Period Begins'
        }]
    }
    fig = dict(data=data, layout=layout)
    
    # 
    # embbeded chart
    #plotly.offline.iplot(fig)
    
    # standalone chart
    plotly.offline.plot(fig)
    
    # online chart 
    # py.iplot(fig, filename='appl-recession-candlestick')

In [94]:
df = load_stockdata('AAPL')
df.head()


,symbol,adj_open,adj_high,adj_low,adj_close,adj_volume,bb_middle,bb_upper,bb_lower,kc_middle,kc_2_upper,kc_2_lower,kc_3_upper,kc_3_lower
effective_date,,,,,,,,,,,,,,
2018-02-15,AAPL,169.790,173.09,169.00,172.99,50609595.0,166.9305,180.703680,153.157320,166.406840,175.938840,156.874840,180.704840,152.108840
2018-02-14,AAPL,163.045,167.54,162.88,167.37,39669178.0,167.2440,181.816823,152.671177,165.713876,172.225876,159.201876,175.481876,155.945876
2018-02-13,AAPL,161.950,164.75,161.65,164.34,32104756.0,167.8305,183.293392,152.367608,165.539547,180.415547,150.663547,187.853547,143.225547
2018-02-12,AAPL,158.500,163.89,157.51,162.71,60560145.0,168.4230,184.210220,152.635780,165.665815,181.019815,150.311815,188.696815,142.634815
2018-02-09,AAPL,157.070,157.89,150.24,155.97,66723743.0,169.1420,185.131502,153.152498,165.976953,178.252953,153.700953,184.390953,147.562953


In [138]:
def multi_chart(df):
    
    style_bollingerbands_middle = dict( color = ('rgb(22, 96, 167)'), width = 1, dash = 'dot')
    style_bollingerbands_upper = dict( color = ('rgb(22, 96, 167)'), width = 1, dash = 'line')
    style_bollingerbands_lower = dict( color = ('rgb(91, 154, 255)'), width = 1, dash = 'line')

    style_kc_middle = dict( color = ('rgb(247, 94, 39)'), width = 1, dash = 'dot')
    style_kc2_upper = dict( color = ('rgb(247, 94, 39)'), width = 1, dash = 'line')
    style_kc2_lower = dict( color = ('rgb(244, 118, 73)'), width = 1, dash = 'line')

    style_kc3_upper = dict( color = ('rgb(46, 155, 53)'), width = 1, dash = 'line')
    style_kc3_lower = dict( color = ('rgb(68, 206, 77)'), width = 1, dash = 'line')

    trace_candle = go.Candlestick(x=df.index, open=df.adj_open, high=df.adj_high, low=df.adj_low, close=df.adj_close, 
                         showlegend=False, name='close price')

    trace_ohlc = go.Ohlc(x=df.index, open=df.adj_open, high=df.adj_high, low=df.adj_low, close=df.adj_close, 
                         showlegend=False, name='close price',
                         increasing=dict(line=dict(color= '#b1e2b6')),
                         decreasing=dict(line=dict(color= '#a89a95'))
                        )
    
    trace_bb_middle = go.Scatter(x=df.index, y=df.bb_middle, name='BB Middle', legendgroup='Bollinger Bands',
                                 showlegend=False, opacity=0.5,
                                 line = style_bollingerbands_middle
                                )
    trace_bb_upper = go.Scatter(x=df.index, y=df.bb_upper, name='BB Upper', legendgroup='Bollinger Bands',
                                line = style_bollingerbands_upper
                               )
    trace_bb_lower = go.Scatter(x=df.index, y=df.bb_lower, name='BB Lower', legendgroup='Bollinger Bands',
                                line = style_bollingerbands_lower
                               )
    
    trace_kc_middle = go.Scatter(x=df.index, y=df.kc_middle, name='KC Middle', legendgroup='Keltner Channels 2ATR',
                                 showlegend=False, opacity=0.5,
                                 line = style_kc_middle
                                )
    trace_kc_2atr_upper = go.Scatter(x=df.index, y=df.kc_2_upper, name='KC 2ATR Upper', legendgroup='Keltner Channels 2ATR',
                                     line = style_kc2_upper
                                    )
    trace_kc_2atr_lower = go.Scatter(x=df.index, y=df.kc_2_lower, name='KC 2ATR Lower', legendgroup='Keltner Channels 2ATR',
                                     line = style_kc2_lower
                                    )

    trace_kc_3atr_upper = go.Scatter(x=df.index, y=df.kc_3_upper, name='KC 3ATR Upper', legendgroup='Keltner Channels 3ATR',
                                     line = style_kc3_upper
                                    )
    trace_kc_3atr_lower = go.Scatter(x=df.index, y=df.kc_3_lower, name='KC 3ATR Lower', legendgroup='Keltner Channels 3ATR',
                                     line = style_kc3_lower
                                    )

    # create volume bar chart on the bottom
    tarce_volume = go.Bar(x=df.index, y=df.adj_volume, name='volume', showlegend=False, marker=dict(color='rgb(87, 91, 130)'),
                          yaxis = 'y2'
                         )
    
    data = [trace_ohlc, trace_bb_middle, trace_bb_upper, trace_bb_lower, 
            trace_kc_middle, trace_kc_2atr_upper, trace_kc_2atr_lower, trace_kc_3atr_upper, trace_kc_3atr_lower, tarce_volume]
    
    layout = go.Layout(
        title = 'John Carter Squeeze',
        xaxis = dict(rangeslider = dict(visible = False)),
        yaxis = dict(domain=[0.2,1]),
        yaxis2 = dict(domain=[0,0.2], visible=False),
        legend = dict(x=0.05, y=1.0)
    )

    fig = go.Figure(data=data, layout=layout)
    # standalone chart
    plotly.offline.plot(fig, "multi chart")


In [139]:
multi_chart(df)